In [1]:
import os
import warnings

warnings.filterwarnings('ignore')

import autosklearn.classification
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from datetime import datetime
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree

pd.options.display.max_columns = 999
RANDOM_STATE = 42


MODEL_DIR = os.path.join(
    'results',
    f"automl-feature-reduction-{datetime.strftime(datetime.now(), '%Y-%m-%d-%H:%M:%S')}"
)

os.makedirs(MODEL_DIR)

MODEL_PATH = os.path.join(MODEL_DIR, 'model.joblib')
DATA_FI_PATH = os.path.join(MODEL_DIR, 'data_fi.csv')
DATA_PATH = 'data/final_train.csv'

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer

In [3]:
df = pd.read_csv(DATA_PATH, index_col=0)

x, y = df.drop(columns = ['Activity']), df['Activity']
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=RANDOM_STATE)

In [4]:
inputer = SimpleImputer(strategy='median')

x_train_t = inputer.fit_transform(x_train)
x_test_t = inputer.transform(x_test)

model = LogisticRegression(penalty='l1', solver='liblinear', C = 0.01)
model.fit(x_train_t, y_train)

LogisticRegression(C=0.01, penalty='l1', solver='liblinear')

In [5]:
y_train_hat = model.predict(x_train_t)
y_test_hat = model.predict(x_test_t)

In [6]:
print('Train results')
print(classification_report(y_train, y_train_hat))
print(confusion_matrix(y_train, y_train_hat))
print('-'*40)

Train results
                    precision    recall  f1-score   support

            LAYING       0.99      1.00      0.99      1650
           SITTING       0.90      0.84      0.87      1479
          STANDING       0.86      0.91      0.89      1500
           WALKING       0.92      0.96      0.94      1417
WALKING_DOWNSTAIRS       0.94      0.93      0.94      1047
  WALKING_UPSTAIRS       0.96      0.92      0.94      1221

          accuracy                           0.93      8314
         macro avg       0.93      0.93      0.93      8314
      weighted avg       0.93      0.93      0.93      8314

[[1648    2    0    0    0    0]
 [  20 1238  214    0    2    5]
 [   0  131 1369    0    0    0]
 [   0    0    0 1365   17   35]
 [   0    0    0   67  974    6]
 [   0    0    0   57   40 1124]]
----------------------------------------


In [7]:
print('Test results')
print(classification_report(y_test, y_test_hat))
print(confusion_matrix(y_test, y_test_hat))
print('-'*40)

Test results
                    precision    recall  f1-score   support

            LAYING       0.99      0.99      0.99       517
           SITTING       0.92      0.83      0.87       540
          STANDING       0.84      0.91      0.87       479
           WALKING       0.92      0.94      0.93       483
WALKING_DOWNSTAIRS       0.94      0.93      0.93       341
  WALKING_UPSTAIRS       0.93      0.93      0.93       412

          accuracy                           0.92      2772
         macro avg       0.92      0.92      0.92      2772
      weighted avg       0.92      0.92      0.92      2772

[[514   2   0   0   1   0]
 [  6 446  85   0   0   3]
 [  0  39 438   1   0   1]
 [  0   0   0 456   7  20]
 [  0   0   0  22 316   3]
 [  0   0   0  16  11 385]]
----------------------------------------


In [8]:
fi = pd.DataFrame(model.coef_, columns=x.columns)
fi = fi.replace({0.0: np.nan})
fi = fi.dropna(how = 'all', axis = 'columns')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [9]:
fi

,"angle(tBodyGyroMean,gravityMean)",energy-mean(),"fBodyAcc-bandsEnergy()-1,16","fBodyAcc-bandsEnergy()-1,24.1",fBodyAcc-entropy()-X,fBodyAcc-entropy()-Y,fBodyAcc-kurtosis()-X,fBodyAcc-max()-Y,fBodyAccJerk-meanFreq()-Z,fBodyAccJerk-std()-Y,fBodyAccMag-mad(),fBodyAccMag-skewness(),fBodyBodyAccJerkMag-entropy(),fBodyBodyAccJerkMag-skewness(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroMag-maxInds,"fBodyGyro-bandsEnergy()-1,8.2",fBodyGyro-entropy()-X,fBodyGyro-kurtosis()-Y,fBodyGyro-maxInds-Z,fBodyGyro-meanFreq()-X,fBodyGyro-skewness()-X,fBodyGyro-skewness()-Z,shadow-gravity-angle(),subject,"tBodyAcc-arCoeff()-X,1",tBodyAcc-sma(),tBodyAccJerk-entropy()-X,tBodyAccJerk-iqr()-Y,tBodyAccJerk-min()-Y,tBodyAccJerkMag-entropy(),tBodyAccJerkMag-mad(),tBodyAccMag-std(),"tBodyGyro-arCoeff()-X,4",tBodyGyro-low()-X,tBodyGyro-low()-Y,tBodyGyro-low()-Z,tBodyGyro-max()-X,tBodyGyro-std()-Y,"tBodyGyroJerk-arCoeff()-Z,2","tBodyGyroJerk-arCoeff()-Z,3",tBodyGyroMag-entropy(),"tGravityAcc-arCoeff()-Y,4","tGravityAcc-arCoeff()-Z,3","tGravityAcc-correlation()-X,Y","tGravityAcc-correlation()-X,Z","tGravityAcc-correlation()-Y,Z",tGravityAcc-entropy()-Y,tGravityAcc-max()-X,tGravityAcc-mean()-X,tGravityAcc-mean()-Y,tGravityAcc-mean()-Z,tGravityAccMag-arCoeff()1,tGravityAccMag-mad(),tGravityAccMag-std()
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.023642,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.401616,1.791513,NaN,-0.310054,NaN,NaN,NaN
1,-0.046892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.238975,NaN,NaN,NaN,0.088073,NaN,-0.015784,NaN,NaN,1.216690,NaN,NaN,NaN,0.326814,-0.241254,NaN,-0.018625,NaN,NaN,NaN,0.040963,NaN,NaN,NaN,NaN,NaN,NaN,-0.000188,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.416535,-0.379636,0.112889,NaN,-0.083813,-0.806975,-1.249745,-1.168484,-0.168987,NaN,NaN,NaN
2,NaN,0.001608,NaN,NaN,0.316250,NaN,NaN,NaN,-0.055802,NaN,NaN,-0.060053,NaN,NaN,NaN,NaN,NaN,NaN,0.067166,NaN,-0.689315,-0.082242,0.200950,NaN,-0.003640,-0.348193,0.094129,NaN,NaN,NaN,0.309908,0.806921,NaN,-0.065697,NaN,NaN,-0.001731,NaN,NaN,0.001132,NaN,-0.316471,NaN,NaN,0.348283,-0.089956,NaN,NaN,NaN,-0.866562,1.749903,0.113652,NaN,0.000544,NaN
3,NaN,NaN,1.311167,NaN,NaN,-0.174038,NaN,NaN,NaN,0.887673,NaN,NaN,NaN,-0.136471,-0.016859,-0.417777,NaN,-0.225773,NaN,-0.074249,NaN,0.010297,NaN,0.036902,0.026965,NaN,NaN,-0.462004,-0.503538,0.125473,-0.519596,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.171318,NaN,-0.070248,-0.619935,0.348106,NaN,NaN,-0.030240,0.081311,NaN,NaN,NaN,NaN,NaN,-0.438520,NaN,0.733276
4,-0.137541,NaN,-0.725629,0.132654,NaN,NaN,-0.042249,NaN,NaN,NaN,-0.195891,NaN,NaN,-0.027559,NaN,NaN,NaN,NaN,0.048277,NaN,NaN,-0.083264,0.651072,0.064999,0.024964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.188776,NaN,NaN,NaN,-0.039747,NaN,0.095957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.110647,NaN,-1.441704
5,0.129271,0.013038,NaN,NaN,-0.077955,NaN,-0.013325,-0.19503,0.888442,NaN,NaN,NaN,NaN,0.277532,NaN,NaN,-0.14525,NaN,-0.332513,0.005346,NaN,-0.113380,-0.475717,-0.155651,-0.021880,0.062832,-0.020734,-0.818088,NaN,NaN,NaN,NaN,NaN,0.000932,0.014847,NaN,0.083688,-0.065879,-0.063706,NaN,NaN,NaN,-0.918716,NaN,NaN,0.005036,NaN,NaN,NaN,NaN,NaN,0.432575,0.002432,NaN,NaN


In [10]:
fi.shape

(6, 55)

In [11]:
x_selected = x[fi.columns]

pd.concat([pd.DataFrame(y, columns = [y.name]), x_selected], axis = 1).to_csv(DATA_FI_PATH, index=False)

In [12]:
df_fi = pd.read_csv(DATA_FI_PATH)

x, y = df_fi.drop(columns = ['Activity']), df_fi['Activity']
x_train_s, x_test_s, y_train_s, y_test_s = train_test_split(x, y, random_state=RANDOM_STATE)

In [13]:
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task = 120
)
automl.fit(x_train_s, y_train_s)

joblib.dump(automl, MODEL_PATH)

y_train_hat = automl.predict(x_train_s)
y_test_hat = automl.predict(x_test_s)

In [14]:
print('Train results')
print(classification_report(y_train_s, y_train_hat))
print(confusion_matrix(y_train_s, y_train_hat))
print('-'*40)

Train results
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00      1650
           SITTING       0.99      1.00      1.00      1479
          STANDING       1.00      0.99      1.00      1500
           WALKING       1.00      1.00      1.00      1417
WALKING_DOWNSTAIRS       1.00      1.00      1.00      1047
  WALKING_UPSTAIRS       1.00      1.00      1.00      1221

          accuracy                           1.00      8314
         macro avg       1.00      1.00      1.00      8314
      weighted avg       1.00      1.00      1.00      8314

[[1650    0    0    0    0    0]
 [   0 1476    2    0    0    1]
 [   0    9 1491    0    0    0]
 [   0    0    0 1417    0    0]
 [   0    0    0    0 1044    3]
 [   0    0    0    0    0 1221]]
----------------------------------------


In [15]:
print('Test results')
print(classification_report(y_test_s, y_test_hat))
print(confusion_matrix(y_test_s, y_test_hat))
print('-'*40)

Test results
                    precision    recall  f1-score   support

            LAYING       1.00      1.00      1.00       517
           SITTING       0.98      0.97      0.98       540
          STANDING       0.97      0.98      0.97       479
           WALKING       1.00      1.00      1.00       483
WALKING_DOWNSTAIRS       0.99      1.00      0.99       341
  WALKING_UPSTAIRS       1.00      0.99      0.99       412

          accuracy                           0.99      2772
         macro avg       0.99      0.99      0.99      2772
      weighted avg       0.99      0.99      0.99      2772

[[516   1   0   0   0   0]
 [  0 525  15   0   0   0]
 [  0  10 469   0   0   0]
 [  0   0   0 483   0   0]
 [  0   0   0   0 340   1]
 [  0   0   0   0   5 407]]
----------------------------------------


In [16]:
automl.get_models_with_weights()

[(0.3,
  SimpleClassificationPipeline({'balancing:strategy': 'weighting', 'classifier:__choice__': 'gradient_boosting', 'data_preprocessing:categorical_transformer:categorical_encoding:__choice__': 'one_hot_encoding', 'data_preprocessing:categorical_transformer:category_coalescence:__choice__': 'minority_coalescer', 'data_preprocessing:numerical_transformer:imputation:strategy': 'mean', 'data_preprocessing:numerical_transformer:rescaling:__choice__': 'robust_scaler', 'feature_preprocessor:__choice__': 'no_preprocessing', 'classifier:gradient_boosting:early_stop': 'train', 'classifier:gradient_boosting:l2_regularization': 2.506856350040198e-06, 'classifier:gradient_boosting:learning_rate': 0.04634380160611007, 'classifier:gradient_boosting:loss': 'auto', 'classifier:gradient_boosting:max_bins': 255, 'classifier:gradient_boosting:max_depth': 'None', 'classifier:gradient_boosting:max_leaf_nodes': 11, 'classifier:gradient_boosting:min_samples_leaf': 41, 'classifier:gradient_boosting:scorin